In [21]:
import findspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import os

In [22]:
findspark.init()
spark = (SparkSession
        .builder
        .appName("transformations")
        .master("local[*]")
        .getOrCreate()
        )

2023-10-03 20:36:11,886 WARN util.Utils: Your hostname, LAPTOP-436V44KT resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wifi0)
2023-10-03 20:36:11,890 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-03 20:36:12,990 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [30]:
"""
Create a bucket (if not exists) in Google Cloud Storage
Download JSON file in bucket (created in preceding step)
"""

import requests
from google.cloud import storage

API_URL = "https://api.covidtracking.com/v2/us/daily.json"
GCP_BUCKET_NAME = "covid19data_ss"
DATA_DESTINATION_NAME = "covidtrackingdata.txt"
SERVICE_ACCOUNT_JSON_PATH = "/home/infernape/gcp-projects/covid19datapipeline/service_account_secrets.json"
PROJECT_ID = "sunlit-vortex-394519"
client = storage.Client.from_service_account_json(SERVICE_ACCOUNT_JSON_PATH)

# Create the bucket if it doesn't exist
def create_bucket_if_not_exists(bucket_name, project_id):
    try:
        client.get_bucket(bucket_name)
        print(f"Bucket {bucket_name} already exists.")
    except Exception as e:
        bucket = client.create_bucket(bucket_name, project=project_id)
        print(f"Bucket {bucket_name} created.")

create_bucket_if_not_exists(GCP_BUCKET_NAME, PROJECT_ID)

# Fetch JSON data from the API
response = requests.get(API_URL)
response.raise_for_status()
data = response.json()["data"]

# Initialize the GCS client with service account credentials
bucket = client.get_bucket(GCP_BUCKET_NAME)
blob = bucket.blob(DATA_DESTINATION_NAME)

# delete JSON file if already exists
if blob.exists():
    blob.delete()
    print(f"Blob {DATA_DESTINATION_NAME} deleted from {GCP_BUCKET_NAME}.")

# Save the JSON data to GCS
blob.upload_from_string(
    data=str(data)
)

print(f"JSON data saved to gs://{GCP_BUCKET_NAME}/{DATA_DESTINATION_NAME}")

Bucket covid19data_ss already exists.
JSON data saved to gs://covid19data_ss/covidtrackingdata.txt


In [34]:
"""
Read downloaded JSON file from GCS bucket
Convert into Spark dataframe and perform transformations
"""

# JSON_FILE_PATH = "/home/infernape/gcp-projects/covid19datapipeline/test/covidtrackingdata.json"

# df = spark.read.json(JSON_FILE_PATH)

INPUT_FILE = "/home/infernape/gcp-projects/covid19datapipeline/test/test.txt"
OUTPUT_FILE = "/home/infernape/gcp-projects/covid19datapipeline/test/covidtrackingdata.json"

rdd = spark.sparkContext.textFile(INPUT_FILE)

def replace_content(line):
    line = line.replace("'", '"')  # Convert single quotes to double quotes
    line = line.replace(": None", ": null")  # Convert None to null
    return line

processed_rdd = rdd.map(replace_content)
df = spark.read.json(processed_rdd)
df.show(1)
# df.write.json(OUTPUT_FILE)

Py4JJavaError: An error occurred while calling o63.json.
: java.net.ConnectException: Call From LAPTOP-436V44KT/127.0.1.1 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1577)
	at org.apache.hadoop.ipc.Client.call(Client.java:1519)
	at org.apache.hadoop.ipc.Client.call(Client.java:1416)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Invok

In [ ]:
from google.cloud import dataproc_v1 as dataproc
from google.cloud import storage
from time import sleep

# TODO: Replace these with your values
PROJECT_ID = 'sunlit-vortex-394519'
REGION = 'us-central1'
CLUSTER_NAME = 'covidprocess'
GCS_BUCKET = 'covid19data_ss'
GCS_INPUT_PATH = f'gs://{GCS_BUCKET}/covidtrackingdata.txt'
GCS_OUTPUT_PATH = f'gs://{GCS_BUCKET}/output.json'
SERVICE_ACCOUNT_JSON_PATH = 'path_to_service_account_key.json'

# Initialize Dataproc and Storage clients
cluster_client = dataproc.ClusterControllerClient()
job_client = dataproc.JobControllerClient()

# Create cluster config
cluster_config = {
    'project_id': PROJECT_ID,
    'cluster_name': CLUSTER_NAME,
    'config': {
        'master_config': {
            'num_instances': 1,
            'machine_type_uri': 'n1-standard-1'
        },
        'worker_config': {
            'num_instances': 0
        },
        'gce_cluster_config': {
            'service_account_scopes': [
                'https://www.googleapis.com/auth/cloud-platform'
            ]
        }
    }
}

# Create cluster
print("Creating cluster...")
cluster = cluster_client.create_cluster(PROJECT_ID, REGION, cluster_config)
cluster_id = cluster.cluster_uuid

# Ensure cluster is up before submitting a job
while True:
    cluster_info = cluster_client.get_cluster(PROJECT_ID, REGION, CLUSTER_NAME)
    if cluster_info.status.state.name == 'RUNNING':
        break
    sleep(10)

# PySpark script to process data
pyspark_script = f"""
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ProcessingData").getOrCreate()

# Read data
rdd = spark.sparkContext.textFile("{GCS_INPUT_PATH}")

# Process data
processed_rdd = rdd.map(lambda x: x.replace("'", '"').replace("None", "null"))

# Convert RDD to DataFrame (assuming each line is valid JSON after transformation)
df = spark.read.json(processed_rdd)

# Save as JSON
df.write.json("{GCS_OUTPUT_PATH}")

spark.stop()
"""

# Save PySpark script to GCS
gcs_client = storage.Client.from_service_account_json(SERVICE_ACCOUNT_JSON_PATH)
bucket = gcs_client.bucket(GCS_BUCKET)
blob = bucket.blob('script/process_data.py')
blob.upload_from_string(pyspark_script)

# Submit job to Dataproc
print("Submitting job...")
job_config = {
    'placement': {
        'cluster_name': CLUSTER_NAME
    },
    'pyspark_job': {
        'main_python_file_uri': f'gs://{GCS_BUCKET}/script/process_data.py'
    }
}
job = job_client.submit_job_as_operation(PROJECT_ID, REGION, job_config)
job_id = job.name

# Wait for job completion
while True:
    job_info = job_client.get_job(PROJECT_ID, REGION, job_id)
    if job_info.status.state.name == 'DONE':
        break
    sleep(10)

# Delete the cluster
print("Deleting cluster...")
cluster_client.delete_cluster(PROJECT_ID, REGION, CLUSTER_NAME)
print("Cluster deleted.")
